# Introduction To VM Protection - VMZeus
> Reverse Engineering VM obfuscation 

- toc: true 
- badges: true
- categories: [vmzues,zeus,vm,obfuscation,tutorial]

## Overview

We are going to take a look at one of the original "VM" protections used in malware, VM Zeus. Zeus is one of the orignal "botnets" that put ecrime on the map. The sourcecode was sold (and eventually leaked) which lead to many different variants. One of the variants implemented a custom VM that was used to protect the configuration file... known as VM Zeus (KINS).

Many of the Zeus variants have been catalogued in the the [ZeusMuseum](https://zeusmuseum.com/) curated by [@tildedennis](https://twitter.com/tildedennis).

In this analysis we are going to focus on the VM mechanics, how it works, and how to analyze it, as a way to understand VM protection fundamentals. There is a lot of prior research on this VM that we will reference below.


### Sample
`f792997cb36a477fa55102ad6b680c97e3517b2e63c83c802bf8d57ae9ed525e` [UnpacMe](https://www.unpac.me/results/bb557f46-a12a-4737-a638-787f982963fd?hash=f792997cb36a477fa55102ad6b680c97e3517b2e63c83c802bf8d57ae9ed525e#/)

### References
- [ZeuS meets VM (BotConf 2014)](https://repo.zenk-security.com/Virus-Infections-Detections-Preventions/Botconf-ZeuS-Meets-VM-Story-so-Far.pdf)
- PNX -- can't find references to his work?
- [ZeusVM analysis - MIASM](https://miasm.re/blog/2016/09/03/zeusvm_analysis.html)
- [Advanced Binary Deobfuscation Course](https://github.com/rung/ABD)
- [Not directly related but interesting work on building a VM -- RISC-Y Business](https://secret.club/2023/12/24/riscy-business.html)

## Analysis 
The VM consists of a dispatcher, a set of instructions handlers, a constant byte array with the **code** and a constant byte array with the **data**.

```c
void __usercall mw_vm(char *arg_data@<esi>)
{
  byte *var_code_buff; // edi
  ctxt ctxt; // [esp+4h] [ebp-4Ch] BYREF

  var_code_buff = mw_cpy_to_buff(&g_code, (void *)0x1000);
  if ( var_code_buff )
  {
    mw_wrap_cpy(arg_data, &g_data, 0x370u);
    ctxt.value0 = 0;
    ctxt.data = (byte *)arg_data;
    ctxt.code = var_code_buff;
    g_data_buff = (int)arg_data;
    while ( ((char (__thiscall *)(ctxt *))ptr_instructions[*ctxt.code])(&ctxt) )
      ;
    mw_wrap_heap_free(var_code_buff);
  }
}
```

### Instruction Set Archetecture

- mnemonic is 1 byte
- operands can be immediate or register index
- regsiters are numerically indexed from 0-15

### Instruction Handlers

- next mnemonic is decrypted using unique hard-coded byte for each handler, and last byte of previous instruction
- code buffer is incremented in handler

### Data
There is a shared data buffer that is pre-populated with data but is also used like stack during instruction execution. Unclear what to call this.

### Context
The VM context is used to pass the code buffer, data buffer, and registers to the instruction handlers. The `value0` is likely flags but not confirmed yet!
```c
struct __declspec(align(4)) ctxt
{
  byte *code;
  byte *data;
  DWORD value0;
  DWORD regs[16];
};
```

